In [1]:
# @title
!pip install transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
import pandas as pd
file_path = "/content/A.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,src,tgt
0,1,"After passing tenth, someone will give you her...",दस ले पास है जाने जब्बए क्वे तुकु अपुडी चैली दीन।
1,2,"Who knows what is in my fate, if life has some...","को जाणूँ क्ये छ म्या्र भाग में, अगर जिबन्ती के..."
2,3,Grandfather took the money and tied it in his ...,बुबुल ऊँ पैंस लि बेर अपण धोतिक गा्ँठ पाड़ लीं।
3,4,"Unite the old village into one, you are a wise...",पुर गौं एक कर राखौ तुम द्वि शैणि मैसनैल।
4,5,We request the readers that we do not lack lit...,पाठकों हैं लै निवेदन छ कि हमार पास साहित्य कि ...


In [3]:
 from transformers import MBartForConditionalGeneration, MBartTokenizer
 model_name = "facebook/mbart-large-50-many-to-many-mmt"
 tokenizer = MBartTokenizer.from_pretrained(model_name)
 model = MBartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [4]:
def tokenize_function(examples):
    tokenizer.src_lang = SRC_LANG
    tokenizer.tgt_lang = TGT_LANG
    # Ensure 'src' and 'tgt' are lists of strings
    src_texts = [str(text) for text in examples["src"]]
    tgt_texts = [str(text) for text in examples["tgt"]]
    model_inputs = tokenizer(src_texts, padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
print(dataset)
print(dataset[0])  # Check a single entry
print(type(dataset[0]["src"]), type(dataset[0]["tgt"]))  # Should be <class 'str'>


Dataset({
    features: ['Unnamed: 0', 'src', 'tgt'],
    num_rows: 1020
})
{'Unnamed: 0': 1, 'src': 'After passing tenth, someone will give you her daughter.', 'tgt': 'दस ले पास है जाने जब्बए क्वे तुकु अपुडी चैली दीन।'}
<class 'str'> <class 'str'>


In [6]:
SRC_LANG = "en_XX"
TGT_LANG = "hi_IN"

In [7]:
encoded_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
 from transformers import AutoTokenizer
 tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [9]:
train_size = int(0.7 * len(encoded_dataset))
train_dataset = encoded_dataset.select(range(train_size))
eval_dataset = encoded_dataset.select(range(train_size, len(encoded_dataset)))

In [10]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=8,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=11,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none",
 )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
from transformers import (MBartForConditionalGeneration, MBartTokenizer,
                          TrainingArguments, Trainer, EarlyStoppingCallback)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Prevent overfitting
)

<ipython-input-11-c31294758e38>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.973700,0.828955
2,0.606800,0.759766
3,0.355300,0.781573
4,0.187200,0.858089


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=572, training_loss=0.8486786972392689, metrics={'train_runtime': 1178.6562, 'train_samples_per_second': 4.846, 'train_steps_per_second': 0.971, 'total_flos': 773665327153152.0, 'train_loss': 0.8486786972392689, 'epoch': 4.0})

In [14]:
model.save_pretrained("/content/trained_model")
tokenizer.save_pretrained("/content/trained_model")

('/content/trained_model/tokenizer_config.json',
 '/content/trained_model/special_tokens_map.json',
 '/content/trained_model/sentencepiece.bpe.model',
 '/content/trained_model/added_tokens.json',
 '/content/trained_model/tokenizer.json')

In [20]:
# Load trained model
model = MBartForConditionalGeneration.from_pretrained("/content/trained_model")
tokenizer = MBartTokenizer.from_pretrained("/content/trained_model")

# Example input
source_text = "we should respect our elders"

# Tokenize input
inputs = tokenizer(source_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Generate translation
generated_tokens = model.generate(inputs["input_ids"], max_length=128)
translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

print(f"Translated text: {translated_text}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Translated text: 老人を尊重し


In [19]:
# Install required libraries
!pip install transformers torch gradio speechrecognition soundfile sentencepiece

# Import required packages
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import gradio as gr
import speech_recognition as sr
import soundfile as sf
import tempfile
import os

# -------------------------------
# Load the Base mBART Model and Fine-Tune (Assumed Already Trained)
# -------------------------------
model_name = "facebook/mbart-large-50-many-to-many-mmt"

# Load the base model and tokenizer
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

# ------ Your Training Code Would Go Here ------
# For demonstration, we'll assume your model has been fine-tuned already.

# -------------------------------
# Save the Fine-Tuned Model & Tokenizer in HuggingFace Format
# -------------------------------
model.save_pretrained("/content/trained_model")
tokenizer.save_pretrained("/content/trained_model")
print("Model and Tokenizer saved successfully in /content/trained_model")

# -------------------------------
# Later on... Load the Fine-Tuned mBART Model from the Saved Directory
# -------------------------------
# Instead of using a .pt file, we load using from_pretrained
model = MBartForConditionalGeneration.from_pretrained("/content/trained_model")
tokenizer = MBart50TokenizerFast.from_pretrained("/content/trained_model")
model.eval()

# -------------------------------
# Define Translation Functions
# -------------------------------
def text_translate(text: str) -> str:
    """
    Translate English text to Kumaoni using fine-tuned Hindi decoder.
    """
    tokenizer.src_lang = "en_XX"  # source is English
    encoded = tokenizer(text, return_tensors="pt")

    # Use fine-tuned model to translate (forced bos token to Hindi which was trained as Kumaoni-style)
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )

    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


def audio_translate(audio) -> tuple:
    """
    Accepts audio as a tuple: (audio_data, sample_rate).
    Saves audio to temporary wav file, transcribes it using SpeechRecognition,
    then translates the text.
    """
    if audio is None:
        return "No audio provided.", ""

    # Save audio to a temporary file
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(temp_file.name, audio[0], audio[1])
    temp_file.close()

    recognizer = sr.Recognizer()
    with sr.AudioFile(temp_file.name) as source:
        audio_data = recognizer.record(source)
    try:
        transcribed = recognizer.recognize_google(audio_data)
    except sr.UnknownValueError:
        transcribed = "Could not understand audio."
    except sr.RequestError as e:
        transcribed = f"Error: {e}"

    os.remove(temp_file.name)

    translation = text_translate(transcribed)
    return transcribed, translation

# -------------------------------
# Custom CSS to Mimic Google Translator
# -------------------------------
custom_css = """
body {
    background-color: #f2f2f2;
    font-family: Arial, sans-serif;
}
h1, .title {
    color: #4285F4;
}
.gradio-container {
    border: 1px solid #ccc;
    border-radius: 8px;
    background-color: #fff;
    padding: 20px;
}
textarea, input {
    border: 1px solid #ccc;
    border-radius: 4px;
    padding: 10px;
}
button {
    background-color: #4285F4;
    color: #fff;
    border: none;
    border-radius: 4px;
    padding: 10px 20px;
    cursor: pointer;
}
button:hover {
    background-color: #3367D6;
}
"""



# -------------------------------
# Build the Gradio Interfaces
# -------------------------------
# Text Translation Interface
text_interface = gr.Interface(
    fn=text_translate,
    inputs=gr.components.Textbox(lines=5, label="Enter English Text"),
    outputs=gr.components.Textbox(label="Translated Kumaoni"),
    title="English ➜ Kumaoni Translator (Text)",
    description="Type your English text below and press 'Submit' to translate.",
    css=custom_css
)

# Audio Translation Interface
audio_interface = gr.Interface(
    fn=audio_translate,
    inputs=gr.Audio(sources=["microphone"], type="numpy", label="Record Audio"),
    outputs=[
        gr.Textbox(label="Transcribed Text"),
        gr.Textbox(label="Translated Kumaoni")
    ],
    title="English ➜ Kumaoni Translator (Audio)",
    description="Record your audio. The app will transcribe and then translate the speech.",
    css=custom_css
)


# Combine interfaces as tabbed layout
app = gr.TabbedInterface(
    [text_interface, audio_interface],
    ["Text Translation", "Audio Translation"],
    css=custom_css
)

# Launch the Gradio app
app.launch(share=True)


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model and Tokenizer saved successfully in /content/trained_model
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c00f5bbef9310486c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
